In [2]:
println(s"Current spark version is ${spark.version}")

Intitializing Scala interpreter ...

Spark Web UI available at http://5e7a77d7892b:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1596789452644)
SparkSession available as 'spark'


Current spark version is 3.0.0


In [1]:
!pwd

/home/jovyan/work



In [4]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath: String = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment: org.apache.spark.sql.DataFrame = [label: int, tweet: string]


In [7]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val rf = new RandomForestClassifier()
  .setNumTrees(10)


val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, rf))


import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_d06e03a382ac
hashingTF: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_0ebbcf7e32c1, binary=false, numFeatures=1000
rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_8321e563b056
pipeline: org.apache.spark.ml.Pipeline = pipeline_50506bd299de


In [8]:
val model = pipeline.fit(raw_sentiment)

model: org.apache.spark.ml.PipelineModel = pipeline_50506bd299de


In [9]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [10]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel: org.apache.spark.ml.PipelineModel = pipeline_50506bd299de


In [11]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[10,21,81,1...|[4.55299036458085...|[0.45529903645808...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[121,193,20...|[4.75416868988563...|[0.47541686898856...|       1.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[17,185,188...|[5.20312754134593...|[0.52031275413459...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[191,330,44...|[5.12831417162505...|[0.51283141716250...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[32,162,166...|[4.84391884773274...|[0.48439188477327...|       1.0|
|    0|@

predictionsDF: org.apache.spark.sql.DataFrame = [label: int, tweet: string ... 5 more fields]


In [12]:
// predictionsDF.schema
// predictionsDF.describe()
// predictionsDF.stat
predictionsDF.printSchema()

root
 |-- label: integer (nullable = false)
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

import org.apache.spark.sql.functions._
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4870/0x000000084199f840@5b6efc44,DoubleType,List(Some(class[value[0]: vector])),None,false,true)


In [14]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5447009635419148|
| 0.5245831310114366|
| 0.4796872458654069|
| 0.4871685828374949|
| 0.5156081152267257|
| 0.5115910332738594|
| 0.4736177838331298|
| 0.5382787130347163|
| 0.5115910332738594|
| 0.4997741671177864|
| 0.5115910332738594|
| 0.4806164859443215|
| 0.4915041120214799|
| 0.5246140422895353|
|0.45369293440451486|
|0.45998172219603484|
| 0.5115910332738594|
| 0.5115910332738594|
| 0.5209467108622792|
| 0.5447009635419148|
+-------------------+
only showing top 20 rows

